In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly ins

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/file/d/1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO/view?usp=drive_link

id='1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('googleplaystore.csv')

In [ ]:
# https://drive.google.com/file/d/1wJgpI28LxgXlWxeLfTJE2O89l3dPbekL/view?usp=drive_link

id='1wJgpI28LxgXlWxeLfTJE2O89l3dPbekL'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('googleplaystoreuserreviews.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df1 = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)
df2 = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystoreuserreviews.csv', header=True, inferSchema=True)
rddreviews = df2.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Desde los ejercicios de pandas sabemos que hay una linea con Rating 19, lo cual no es posible, la sacamos

In [ ]:
rddapps = df1.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame").cache()

Vemos un poco como se presentan los datos de cada registro

In [ ]:
rddapps.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating=4.5, Reviews='215644', Size='25M

In [ ]:
rddreviews.take(5)

[Row(App='10 Best Foods for You', Translated_Review='I like eat delicious food. That\'s I\'m cooking food myself, case "10 Best Foods" helps lot, also "Best Before (Shelf Life)"', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.5333333333333333'),
 Row(App='10 Best Foods for You', Translated_Review='This help eating healthy exercise regular basis', Sentiment='Positive', Sentiment_Polarity='0.25', Sentiment_Subjectivity='0.28846153846153844'),
 Row(App='10 Best Foods for You', Translated_Review='nan', Sentiment='nan', Sentiment_Polarity='nan', Sentiment_Subjectivity='nan'),
 Row(App='10 Best Foods for You', Translated_Review='Works great especially going grocery store', Sentiment='Positive', Sentiment_Polarity='0.4', Sentiment_Subjectivity='0.875'),
 Row(App='10 Best Foods for You', Translated_Review='Best idea us', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.3')]

Vemos que hay apps repetidas

In [ ]:
aux = rddapps.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
aux1 = aux.filter(lambda x: x[1] > 1)
aux1.count()

798

En mi criterio, las sacamos

In [ ]:
rddapps1 = rddapps.map(lambda x: (x[0], x)).reduceByKey(lambda x, y: x)
rddapps1.take(5)

[('Photo Editor & Candy Camera & Grid & ScrapBook',
  Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up')),
 ('Coloring book moana',
  Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up')),
 ('U Launcher Lite – FREE Live Cool Themes, Hide Apps',
  Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver=

De las apps unicamente me quedo con el nombre de las apps y su categoria

In [ ]:
rddapps2 = rddapps1.map(lambda x: (x[0], x[1][1]))
rddapps2.take(5)

[('Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN'),
 ('Coloring book moana', 'ART_AND_DESIGN'),
 ('U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN'),
 ('Sketch - Draw & Paint', 'ART_AND_DESIGN'),
 ('Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN')]

De las reviews me quedo unicamente con aquellas que tienen sentimiento negativo

In [ ]:
rddreviews1 = rddreviews.filter(lambda x: x.Sentiment == "Negative")
rddreviews1.take(5)

[Row(App='10 Best Foods for You', Translated_Review='No recipe book Unable recipe book.', Sentiment='Negative', Sentiment_Polarity='-0.5', Sentiment_Subjectivity='0.5'),
 Row(App='10 Best Foods for You', Translated_Review='Waste time It needs internet time n ask calls information', Sentiment='Negative', Sentiment_Polarity='-0.2', Sentiment_Subjectivity='0.0'),
 Row(App='10 Best Foods for You', Translated_Review='Faltu plz waste ur time', Sentiment='Negative', Sentiment_Polarity='-0.2', Sentiment_Subjectivity='0.0'),
 Row(App='10 Best Foods for You', Translated_Review="Crap Doesn't work", Sentiment='Negative', Sentiment_Polarity='-0.8', Sentiment_Subjectivity='0.8'),
 Row(App='10 Best Foods for You', Translated_Review="Boring. I thought actually just texts that's it. Too poor old texts....", Sentiment='Negative', Sentiment_Polarity='-0.32499999999999996', Sentiment_Subjectivity='0.47500000000000003')]

Nos quedamos con el nombre de la app y su cantidad de reviews negativas.

In [ ]:
rddreviews2 = rddreviews1.map(lambda x: (x[0], 1))
rddreviews3 = rddreviews2.reduceByKey(lambda a, b: a + b)
rddreviews3.take(5)

[('104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室', 1),
 ('21-Day Meditation Experience', 10),
 ('2Date Dating App, Love and matching', 7),
 ('30 Day Fitness Challenge - Workout at Home', 2),
 ('4 in a Row', 3)]

Juntamos ambos rdds y mapeamos para tener como key la categoria y como valores el nombre de la app y su cantidad de reviews negativas

In [ ]:
rddcombinado = rddapps2.join(rddreviews3)
rddcombinado1 = rddcombinado.map(lambda x: (x[1][0], (x[0], x[1][1])))
rddcombinado1.take(5)

[('ART_AND_DESIGN', ("Boys Photo Editor - Six Pack & Men's Suit", 5)),
 ('ART_AND_DESIGN', ('Colorfit - Drawing & Coloring', 8)),
 ('ART_AND_DESIGN', ('Easy Origami Ideas', 11)),
 ('ART_AND_DESIGN', ('Canva: Poster, banner, card maker & graphic design', 3)),
 ('AUTO_AND_VEHICLES', ('CDL Practice Test 2018 Edition', 1))]

Por ultimo nos quedamos con la app con mas reviews de sentimiento negativo de cada categoria y hacemos collect ya que sabemos que son pocas.

In [ ]:
rddcombinado2 = rddcombinado1.reduceByKey(lambda a, b: a if a[1] > b[1] else b)
rddrespuesta = rddcombinado2.map(lambda x: (x[0], x[1][0]))
rddrespuesta.collect()

[('COMMUNICATION', 'Azar'),
 ('ENTERTAINMENT', 'Colorfy: Coloring Book for Adults - Free'),
 ('FINANCE', 'Google Pay'),
 ('MEDICAL', 'Banfield Pet Health Tracker'),
 ('SOCIAL', 'Facebook'),
 ('TOOLS', 'Gboard - the Google Keyboard'),
 ('NEWS_AND_MAGAZINES', 'Fox News – Breaking News, Live Video & News Alerts'),
 ('EVENTS', 'DroidAdmin for Android - Advice'),
 ('BEAUTY', 'BestCam Selfie-selfie, beauty camera, photo editor'),
 ('COMICS', 'DC Comics'),
 ('EDUCATION', 'Duolingo: Learn Languages Free'),
 ('LIBRARIES_AND_DEMO', 'Aviary Stickers: Free Pack'),
 ('LIFESTYLE', 'Fashion in Vogue'),
 ('FAMILY', 'Bad Piggies'),
 ('PHOTOGRAPHY', 'Google Photos'),
 ('PARENTING', 'Baby Name Together'),
 ('MAPS_AND_NAVIGATION', 'FindShip'),
 ('ART_AND_DESIGN', 'Coloring book moana'),
 ('AUTO_AND_VEHICLES', 'BEST CAR SOUNDS'),
 ('BOOKS_AND_REFERENCE', 'Amazon Kindle'),
 ('BUSINESS', 'HipChat - Chat Built for Teams'),
 ('DATING', 'Free Dating App - Meet Local Singles - Flirt Chat'),
 ('FOOD_AND_DRINK', '